## Data Conversion

The original source music_v3 relational database is in a MySql database. The destination database is a Mongo Database.

Source data field clean-up methods are in the cell below

In [25]:
import numpy as np
import pandas as pd
import re


def title_cleanup(title):
    """
    Process song titles and return
    cleaned collection (tag), title and notes
    """
    # title beginning characters
    r1 = re.findall(r'^[xX]', title)
    r2 = re.findall(r'^[qQ]', title)
    tag = None
    note = None
    
    # Special Character to represent collection tags
    if r1:
        tag = 'Youth'
        title = re.sub(r'^[xX]', '', title)
    elif r2:
        tag = 'Special'
        title = re.sub(r'^[qQ]', '', title)
    else:
        tag = 'Main'
    
    title = title.strip()
    
    # remove the extra dash seperating special character
    if title.startswith('-'):
        title = title[1 : : ]
        title = title.strip()
    
    # Read notes inside parenthesis and slice off first occurance
    found = re.findall(r'\((.*?) *\)', title)
    if found:
        note = found[0]
        title = title.rpartition('(')[0].strip()
    
    return title, tag, note

def begin_words_cleanup(words):
    """
    Provide the first 4 cleaned words in English and possibly =S(panish
    df - a dataframe with song lyrics
    """
    spanish = None
    english = words.split(' ')[:12] # first 12 words
    english = ' '.join(english)
    english = english.replace('\r', '')
    english = english.replace('\n', ' ')
    # Within parens
    found = re.findall(r'\((.*?) *\)', english)
    
    if found:
        spanish = found[0]
        spanish = ' '.join(spanish.split()[:4])
        # replace parens and content
        english = re.sub(r' ?\([^)]+\)', '', english)
    
    # Limit to 4 words maximum
    english = ' '.join(english.split()[:4])
        
    return english, spanish

def process_songs(df):
    """
    Process rows in the songs dataframe to clean-up and recover data
    df - The source Dataframe
    return the cleaned Dataframe
    """
    for (idx, row) in df.iterrows():
        row_id = row['id']
        title, tag, note = title_cleanup(row.get('title'))
        
        # Change in the actual dataframe.
        df.loc[idx, 'tag'] = tag
        df.loc[idx, 'title'] = title
        df.loc[idx, 'note'] = note

        english, spanish = begin_words_cleanup(row.get('words'))
        df.loc[idx, 'begin_english'] = english
        df.loc[idx, 'begin_spanish'] = spanish
         
    return df

def flatten_list_of_list(lol):
    """Flatten a list of Lists"""
    result = [r for sublist in lol for r in sublist]
    return result
    
def do_language_cleanup(words):
    """
    Seperate mixed, english, and spanish words and return all three.
    """
    english = words
    spanish = None
    
    # Spanish Within parens
    found = re.findall(r'\((.*?) *\)', words)
    
    if found:
        print(found)
        spanish = ' '.join(found)
        # replace parens and content
        english = re.sub(r' ?\([^)]+\)', '', words)
    return words, english, spanish
    
def process_scores_voice(df):
    """
    Process rows in the scores voice dataframe to clean-up and recover data
    df - The source Dataframe
    return the cleaned Dataframe
    """
    for (idx, row) in df.iterrows():
        row_id = row['id']
        
        mixed, english, spanish = do_language_cleanup(row.get('words'))
        df.loc[idx, 'words'] = mixed
        df.loc[idx, 'english'] = english
        df.loc[idx, 'spanish'] = spanish
        
        #  TODO process instrument score images
        
    return df

def cleanup_songs(df):
    """
    Do the full songs source data clean-up
    df - a dataframe with the source songs.
    """
    df = process_songs(df)
    df.drop(columns=['words'], inplace=True)
    print(df.head())
    return df

def cleanup_scores(dfvoice, dfinstrument=None):
    """
    Do the scores source data for the vocal instrument clean-up
    df - a dataframe with the source vocal scores
    """
    df = process_scores_voice(dfvoice)
    print(df.head())
    return df


In [20]:
# Test do_language_cleanup
r0 = do_language_cleanup('Blessing and Honor (Bendición y honor) \r\nGlory and Power (gloria y poder)')
r0

['Bendición y honor', 'gloria y poder']


('Blessing and Honor (Bendición y honor) \r\nGlory and Power (gloria y poder)',
 'Blessing and Honor \r\nGlory and Power',
 'Bendición y honor gloria y poder')

In [39]:
# A test of the title cleanup function
r1 = title_cleanup('Q-MyTitle belongs (my note)')
r1

('MyTitle belongs', 'Special', 'my note')

In [40]:
# A test of the words cleanup method
r2 = begin_words_cleanup('Blessing and Honor (Bendición y honor) Glory and Power (gloria y poder)')
r2

('Blessing and Honor Glory', 'Bendición y honor')

## Query legacy music database

Query for the source music data and then perform the cleanup needed in the methods above

In [28]:
import openpyxl
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import pymysql
from dotenv import dotenv_values

# Read secret values
config = dotenv_values('.env')
SQLALCHEMY_DATABASE_URI = config.get('SQLALCHEMY_DATABASE_URI')
engine = create_engine(SQLALCHEMY_DATABASE_URI, echo=False)


SONGS_SQL = text(
"""
SELECT
  id, 
  tempo_cd_id,
  arrangement,
  NULL as tag,
  title,
  start_key, 
  aux_key,
  words,
  NULL as begin_english,
  NULL as begin_spanish,
  NULL as note
FROM song;
""")

SCORES_SQL = text(
"""
SELECT
  id,
  id as song, 
  8 as instrument,
  words,
  NULL as english,
  NULL as spanish,
  'text/plain' as mime_type
from song;
""")

with engine.connect() as conn:
    songs = pd.read_sql(SONGS_SQL, conn)
    #print(songs.head())
    
    scores = pd.read_sql(SCORES_SQL, conn)
    #print(scores.head())
    


songs = cleanup_songs(songs)
scores = cleanup_scores(scores)


songs.to_excel('songs.xlsx', index=False)
scores.to_excel('scores.xlsx', index=False)



   id  tempo_cd_id                     arrangement   tag  \
0   1            1  Faithful one, so unchanging...  Main   
1   2            4                            None  Main   
2   3            3     Jesus is my Lord and Savior  Main   
3   5            2                            None  Main   
4   7            4                            None  Main   

                         title start_key aux_key               begin_english  \
0                  Ageless One         C    None  Faithful One So unchanging   
1  Ah Lord God /In Him We Live         C    None           Ah, Lord God Thou   
2     You're Alive Forevermore        Dm    None            Jesus is my Lord   
3         All Honor, All Glory        Dm    None       All honor, All glory,   
4       All The Way To Calvary      Eb/F    None              All the way to   

       begin_spanish  note  
0               None  None  
1  !Oh Señor Jehová!  None  
2               None  None  
3               None  None  
4  Todo el ca

['Que grande es nuestro Dios', 'Que grande es Su nombre', 'El es grandisimo', 'Para siempre lo mismo', 'El rodo hacia atras las aguas', 'Del poderoso Mar Rojo', 'Y El dijo, estare contigo', 'Solo confia en mi']
['Rey de reyes', 'Y Señor de señores', 'Gloria Aleluya', 'Rey de reyes', 'Y Señor de señores', 'Gloria Aleluya', 'Jesús, Príncipe de la Paz', 'Gloria Aleluya', 'Te amamos', 'Te amamos', 'Jesús, Jesús']
['Emocionense', 'Ve a decirle a todos que', 'Jesucristo es Rey', 'Emocionense', 've a decirle a todos que', 'Jesuscristo es Rey', 'Emocionense', 've a decirle a todos', 'Jesuscristo es Rey', 'Jesuscristo es', 'todavia el Rey de Reyes', 'Tu hablas de personas', 'tu hablas de cosas', 'que realmente', 'No son importante', 'hablas del clime', 'hablas de problemas', 'que tenemos aqui', 'y en el mundo', 'Amigo dejame decirte', 'que tengo una solucion', 'para este mundo', 'voy a gritar y cantar', 'Jesuscristo todavia', 'es el Rey de Reyes']
['Aleluya Aleluya', 'Nuestro  Dios Todopoderoso

## Define access methods to Destination MongoDB

 
* _connect
* read_mongo - reads from a mongo database collection
* write_mongo - writes a Dataframe into a mongo database collection

In [29]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)

    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)
    
    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

def write_mongo(db, collection, df, host='localhost', port=27017, username=None, password=None, no_id=True):
    """Write a Dataframe to a Mongo Database"""
    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    # Remove previous data
    db[collection].drop()

    # Write the data
    #df.reset_index(inplace=True)
    data = df.to_dict(orient='records')  # Here's our added param..
    db[collection].insert_many(data)
    

## Write Data

* songs into songs collection
* scores into the scores collection

In [30]:
write_mongo('music', 'songs', songs)
write_mongo('music', 'scores', scores)

In [9]:
# Test the songs collection result
sngs = read_mongo('music', 'songs')
sngs.head()

,id,tempo_cd_id,arrangement,tag,title,start_key,aux_key,begin_english,begin_spanish,note
0,1,1,"Faithful one, so unchanging...",Main,Ageless One,C,None,Faithful One So unchanging,None,None
1,2,4,None,Main,Ah Lord God /In Him We Live,C,None,"Ah, Lord God Thou",!Oh Señor Jehová!,None
2,3,3,Jesus is my Lord and Savior,Main,You're Alive Forevermore,Dm,None,Jesus is my Lord,None,None
3,5,2,None,Main,"All Honor, All Glory",Dm,None,"All honor, All glory,",None,None
4,7,4,None,Main,All The Way To Calvary,Eb/F,None,All the way to,Todo el camino al,None


In [31]:
# Test the scores collection result
scr = read_mongo('music', 'scores')
scr.head()

,id,song,instrument,words,english,spanish,mime_type
0,1,1,8,Faithful One\r\nSo unchanging\r\nAgeless One\r...,Faithful One\r\nSo unchanging\r\nAgeless One\r...,None,text/plain
1,2,2,8,"Ah, Lord God\r\n(!Oh Señor Jehová!) \r\nThou h...","Ah, Lord God\r\n \r\nThou hast made the heaven...",!Oh Señor Jehová! He aquí que tú hiciste el ci...,text/plain
2,3,3,8,"Jesus is my\r\nLord and savior,\r\nSent here t...","Jesus is my\r\nLord and savior,\r\nSent here t...",None,text/plain
3,5,5,8,"All honor,\r\nAll glory,\r\nAll power to You.\...","All honor,\r\nAll glory,\r\nAll power to You.\...",None,text/plain
4,7,7,8,All the way to Calvary \r\n(Todo el camino al ...,"All the way to Calvary \r\n\r\nHe went for me,...",Todo el camino al Calvario Él fue por mí Él fu...,text/plain
